<a href="https://colab.research.google.com/github/rosalieCRX/UnemploytmentRatePython/blob/main/ADL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import statsmodels.api as sm

def lag(series, lag):
  if lag == 0:
    return series
  else:
    return series.shift(lag)

def all_lags(series,step = 1, lagNum = 12,changeName = False, Name = 'Y'):
  lags = {}
  for i in range(step,lagNum+step):
    x = lag(series,i)
    if changeName:
      lags['lag %i-%s'%(i,Name)] = x
    else:
      lags['lag %i'%i] = x

  return DataFrame(lags)


def add_seasonality(df):
  sea = {1:'JANUARY',2:'FEBRUARY',3:'MARCH',4:'APRIL',5:'MAY',6:'JUNE',7:'JULY',8:'AUGUST',9:'SEPTEMBER',10:'OCTOBER',11:'NOVEMBER',12:'DECEMBER'}
  season = {'JANUARY':[],'FEBRUARY':[],'MARCH':[],'APRIL':[],'MAY':[],'JUNE':[],'JULY':[],'AUGUST':[],'SEPTEMBER':[],'OCTOBER':[],'NOVEMBER':[],'DECEMBER':[]}
  for index, row in df.iterrows():
    for i in season:
      if i == sea[index.month]:
        season[i].append(1)
      else:
        season[i].append(0)
  season = pandas.DataFrame.from_dict(season)
  season.index = df.index.copy()
  return  season
#columns = ['JANUARY','FEBRUARY','MARCH','APRIL','MAY','JUNE','JULY','AUGUST','SEPTEMBER','OCTOBER','NOVEMBER','DECEMBER'],



def print_AIC_grid(grid):
  for i in grid:
    print(f'{i: <30}     AIC: {round(grid[i][0],2): <10}  # of Lags: {grid[i][1]}')


def AIC_grid(y,x):
  y_lags = all_lags(y)
  y_lags = pandas.concat([y_lags, add_seasonality(y_lags)], axis=1)

  print(f'{bcolors.OKGREEN }>>> seasonality added{bcolors.ENDC}')

  AIC_grid = {}
  AIC_grid['Unemployment Rate'] = []
  print('>>> Selecting # of lags for dependent variable:\n')
  for i in reversed(range(1,13)):
    reg_result = sm.OLS(y[588:-1], y_lags[588:-1]).fit()
    # print(reg_result.summary())
    AIC_grid['Unemployment Rate'].append((reg_result.aic,i))
    name = "lag %i"%i
    y_lags.drop(name, axis=1, inplace=True)
  # print(*AIC_grid['Unemployment Rate'], sep = "\n")
  AIC_grid['Unemployment Rate'] = sorted(AIC_grid['Unemployment Rate'])[0]
  print_AIC_grid(AIC_grid)

  # get optimum dependent & lagged dependent series
  y_opt = AIC_grid['Unemployment Rate'][1]
  y_opt = all_lags(y,lagNum = y_opt,True)



  print('\n\n>>> Selecting # of lags for leading indicators:\n')
  for column in x:
    x_lags = all_lags(column)
    x_lags = pandas.concat([x_lags, y_opt], axis=1)
    x_lags = pandas.concat([x_lags, add_seasonality(x_lags)], axis=1)
    AIC_grid[column.name] = []
    for i in reversed(range(1,13)):
      reg_result = sm.OLS(y[588:-1], x_lags[len(x_lags)-len(y)+588:-1]).fit()
      # print(reg_result.summary())
      AIC_grid[column.name].append((reg_result.aic,i))
      name = "lag %i"%i
      x_lags.drop(name, axis=1, inplace=True)
    # print(*AIC_grid[column.name], sep = "\n")
    AIC_grid[column.name] = sorted(AIC_grid[column.name])[0]
  AIC_grid = dict(sorted(AIC_grid.items(), key=lambda item: item[1]))
  print_AIC_grid(AIC_grid)
  

def getForecastModel(df,y_name,lag_per_series,step=1,seasonality = False):
  # get y, lagged y, and lagged x
  y = df[y_name]
  x = all_lags(df[y_name],lagfrom = step, lagNum = lag_per_series[y_name],True)
  for series_name in lag_per_series:
    x_lags = all_lags(df[series_name],lagNum=lag_per_series[series_name],True)
    x = pandas.concat([x, x_lags], axis=1)
  if seasonality:
    x = pandas.concat([x_lags, add_seasonality(x_lags)], axis=1)
  return y,x

def getForecastData(df,step = 1,lag_per_series):
  data = {}
  for i in lag_per_series:
    for j in range(1,lag_per_series[i]+1):
      data['lag %i-%s'%(j+step-1,i)] = df.loc[df.index[-1*j],i]
  
  data = pandas.DataFrame.from_dict(data)
  data.set_index([df.index[-1]],inplace = True)
  if seasonality:
    data = pandas.concat([data, add_seasonality(data)], axis=1)
  return data
  
    
def forecast(df,y_name,lag_per_series,step=1,seasonality = False):
  x = getForecastData(df,step,lag_per_series)
  





In [ ]:
y = df['Unemployment Rate']
x = [df[i] for i in df if i!= 'Unemployment Rate']
AIC_grid(y,x)



print('\n\n>>> Use several leading indicators:')
y_opt = all_lags(y,lagNum=10,True)
Corporate_Spread = all_lags(df['Corporate Spread'],lagNum=2,True,'CS')
CLI  = all_lags(df['CLI'],lagNum=3,True,'CLI')
IC = all_lags(df['Initial Claims'],lagNum=5,True,'IC')

cs_cli_ic = Corporate_Spread.copy()
cs_cli_ic = pandas.concat([cs_cli_ic, CLI], axis=1)
cs_cli_ic = pandas.concat([cs_cli_ic, IC], axis=1)
cs_cli_ic = pandas.concat([cs_cli_ic, y_opt], axis=1)
cs_cli_ic = pandas.concat([cs_cli_ic, add_seasonality(cs_cli_ic)], axis=1)

cs_cli = Corporate_Spread.copy()
cs_cli = pandas.concat([cs_cli, CLI], axis=1)
cs_cli = pandas.concat([cs_cli, y_opt], axis=1)
cs_cli = pandas.concat([cs_cli, add_seasonality(cs_cli)], axis=1)

cs_ic = Corporate_Spread.copy()
cs_ic = pandas.concat([cs_ic, IC], axis=1)
cs_ic = pandas.concat([cs_ic, y_opt], axis=1)
cs_ic = pandas.concat([cs_ic, add_seasonality(cs_ic)], axis=1)

cli_ic = CLI.copy()
cli_ic = pandas.concat([cli_ic, IC], axis=1)
cli_ic = pandas.concat([cli_ic, y_opt], axis=1)
cli_ic = pandas.concat([cli_ic, add_seasonality(cli_ic)], axis=1)

AIC_grid={}
AIC_grid['Corporate_Spread + cli + Initial Claims'] = sm.OLS(y[588:-1], cs_cli_ic[len(cs_cli_ic)-len(y)+588:-1]).fit().aic
AIC_grid['Corporate_Spread + cli'] = sm.OLS(y[588:-1], cs_ic[len(cs_ic)-len(y)+588:-1]).fit().aic
AIC_grid['cli + Initial Claims'] = sm.OLS(y[588:-1], cli_ic[len(cli_ic)-len(y)+588:-1]).fit().aic
print(AIC_grid)


NameError: ignored